In [121]:
import pandas as pd
import numpy as np
import streamlit as st

In [122]:
brasileirao = pd.read_excel('./data/brasileirao2023.xlsx')
brasileirao

,Temporada,Rodada,Mandante,Score_m,Visitante,Score_v
0,2023,1,PAL,2.0,CUI,1.0
1,2023,1,AME,0.0,FLU,3.0
2,2023,1,CAP,2.0,GOI,0.0
3,2023,1,BOT,2.0,SAO,1.0
4,2023,1,RED,2.0,BAH,1.0
...,...,...,...,...,...,...
375,2023,38,GOI,NaN,AME,NaN
376,2023,38,SAN,NaN,FOR,NaN
377,2023,38,SAO,NaN,FLA,NaN
378,2023,38,VAS,NaN,RED,NaN


In [123]:
brasileirao = brasileirao[brasileirao['Score_m'].notnull()]
brasileirao

,Temporada,Rodada,Mandante,Score_m,Visitante,Score_v
0,2023,1,PAL,2.0,CUI,1.0
1,2023,1,AME,0.0,FLU,3.0
2,2023,1,CAP,2.0,GOI,0.0
3,2023,1,BOT,2.0,SAO,1.0
4,2023,1,RED,2.0,BAH,1.0
...,...,...,...,...,...,...
195,2023,20,GOI,1.0,CAP,1.0
196,2023,20,SAN,2.0,GRE,1.0
197,2023,20,VAS,1.0,CAM,0.0
198,2023,20,SAO,0.0,BOT,0.0


In [124]:
times = brasileirao['Mandante'].unique()
tabela = pd.DataFrame()
for time in times:
    vit = emp = der = pro = con = jog = 0
    # emp = 0
    # der = 0
    # pro = 0
    # con = 0
    # jog = 0
    mandante = brasileirao[brasileirao['Mandante'] == time]
    for indice, partida in mandante.iterrows():
        if partida['Score_m'] > partida['Score_v']:
            vit += 1 
        elif partida['Score_m'] == partida['Score_v']:
            emp += 1
        else:
            der += 1
        pro += partida['Score_m']
        con += partida['Score_v']
        jog += 1
    visitante = brasileirao[brasileirao['Visitante'] == time]
    for indice, partida in visitante.iterrows():
        if partida['Score_v'] > partida['Score_m']:
            vit += 1
        elif partida['Score_v'] == partida['Score_m']:
            emp += 1
        else:
            der += 1
        pro += partida['Score_v']
        con += partida['Score_m']
        jog += 1
    new_row = pd.DataFrame({
        '1-Time': [time],
        '3-J': [jog],
        '4-V': [vit],
        '5-E' : [emp],
        '6-D': [der],
        '7-GP': [pro],
        '8-GC': [con]
    })   
    tabela = pd.concat([tabela, new_row], ignore_index = True)
    
tabela['2-P'] = (tabela['4-V'] * 3) + tabela['5-E']
tabela['9-SG'] = tabela['7-GP'] - tabela['8-GC']   
tabela_sort = tabela.sort_values(by=['2-P','4-V','9-SG'], ascending=False)
tabela_sort

,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG
3,BOT,20,15,3,2,35.0,11.0,48,24.0
0,PAL,20,10,7,3,36.0,17.0,37,19.0
8,FLA,20,10,5,5,34.0,26.0,35,8.0
10,FLU,20,10,4,6,28.0,20.0,34,8.0
9,GRE,20,10,4,6,29.0,25.0,34,4.0
2,CAP,20,9,5,6,29.0,23.0,32,6.0
4,RED,20,8,8,4,27.0,21.0,32,6.0
5,FOR,20,8,5,7,22.0,19.0,29,3.0
11,CUI,20,8,4,8,21.0,23.0,28,-2.0
12,SAO,20,7,7,6,24.0,19.0,28,5.0


In [125]:
tabela

,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG
0,PAL,20,10,7,3,36.0,17.0,37,19.0
1,AME,20,2,5,13,20.0,44.0,11,-24.0
2,CAP,20,9,5,6,29.0,23.0,32,6.0
3,BOT,20,15,3,2,35.0,11.0,48,24.0
4,RED,20,8,8,4,27.0,21.0,32,6.0
5,FOR,20,8,5,7,22.0,19.0,29,3.0
6,CAM,20,7,6,7,22.0,18.0,27,4.0
7,COR,20,6,7,7,21.0,22.0,25,-1.0
8,FLA,20,10,5,5,34.0,26.0,35,8.0
9,GRE,20,10,4,6,29.0,25.0,34,4.0


In [126]:
from sklearn.cluster import KMeans
df_data = tabela[[
    '2-P',              
    '4-V',
    '5-E',
    '6-D',
    '9-SG'
]]
kmeans = KMeans(n_clusters=5, random_state=0).fit(df_data)
tabela['cluster'] = kmeans.labels_
# tabela['c_p'] = kmeans.cluster_centers_[kmeans.labels_,0]
# tabela['c_v'] = kmeans.cluster_centers_[kmeans.labels_,1]
# tabela['c_e'] = kmeans.cluster_centers_[kmeans.labels_,2]
# tabela['c_d'] = kmeans.cluster_centers_[kmeans.labels_,3]
# tabela['c_sg'] = kmeans.cluster_centers_[kmeans.labels_,4]
tabela

c:\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG,cluster
0,PAL,20,10,7,3,36.0,17.0,37,19.0,3
1,AME,20,2,5,13,20.0,44.0,11,-24.0,0
2,CAP,20,9,5,6,29.0,23.0,32,6.0,1
3,BOT,20,15,3,2,35.0,11.0,48,24.0,3
4,RED,20,8,8,4,27.0,21.0,32,6.0,1
5,FOR,20,8,5,7,22.0,19.0,29,3.0,4
6,CAM,20,7,6,7,22.0,18.0,27,4.0,4
7,COR,20,6,7,7,21.0,22.0,25,-1.0,4
8,FLA,20,10,5,5,34.0,26.0,35,8.0,1
9,GRE,20,10,4,6,29.0,25.0,34,4.0,1


In [127]:
print(kmeans.labels_)

[3 0 1 3 1 4 4 4 1 1 1 4 4 4 2 0 2 0 2 2]


In [128]:
kmeans.cluster_centers_

array([[ 14.  ,   3.  ,   5.  ,  12.  , -19.  ],
       [ 33.4 ,   9.4 ,   5.2 ,   5.4 ,   6.4 ],
       [ 22.25,   5.5 ,   5.75,   8.75,  -7.5 ],
       [ 42.5 ,  12.5 ,   5.  ,   2.5 ,  21.5 ],
       [ 27.  ,   7.  ,   6.  ,   7.  ,   2.  ]])

In [129]:
df_cluster = pd.DataFrame()
for cluster, colunas in enumerate(kmeans.cluster_centers_):
    print(cluster)
    print(colunas)
    new_row = pd.DataFrame({
        'cluster': [cluster], 
        '2-P': [colunas[0]],              
        '4-V': [colunas[1]],
        '5-E': [colunas[2]],
        '6-D': [colunas[3]],
        '9-SG': [colunas[4]],
    })
    df_cluster = pd.concat([df_cluster, new_row], ignore_index=True)
df_cluster

0
[ 14.   3.   5.  12. -19.]
1
[33.4  9.4  5.2  5.4  6.4]
2
[22.25  5.5   5.75  8.75 -7.5 ]
3
[42.5 12.5  5.   2.5 21.5]
4
[27.  7.  6.  7.  2.]


,cluster,2-P,4-V,5-E,6-D,9-SG
0,0,14.00,3.0,5.00,12.00,-19.0
1,1,33.40,9.4,5.20,5.40,6.4
2,2,22.25,5.5,5.75,8.75,-7.5
3,3,42.50,12.5,5.00,2.50,21.5
4,4,27.00,7.0,6.00,7.00,2.0


In [130]:
df_cluster_sort = df_cluster.sort_values(by='2-P', ascending=False)
df_cluster_sort

,cluster,2-P,4-V,5-E,6-D,9-SG
3,3,42.50,12.5,5.00,2.50,21.5
1,1,33.40,9.4,5.20,5.40,6.4
4,4,27.00,7.0,6.00,7.00,2.0
2,2,22.25,5.5,5.75,8.75,-7.5
0,0,14.00,3.0,5.00,12.00,-19.0


In [131]:
df_cluster_sort['grupo'] = ['Título','Libertadores','Sul-Americana','Limbo','Rebaixamento']
df_cluster_sort

,cluster,2-P,4-V,5-E,6-D,9-SG,grupo
3,3,42.50,12.5,5.00,2.50,21.5,Título
1,1,33.40,9.4,5.20,5.40,6.4,Libertadores
4,4,27.00,7.0,6.00,7.00,2.0,Sul-Americana
2,2,22.25,5.5,5.75,8.75,-7.5,Limbo
0,0,14.00,3.0,5.00,12.00,-19.0,Rebaixamento


In [132]:
df_cluster_grupo = df_cluster_sort[['cluster','grupo']]
df_cluster_grupo

,cluster,grupo
3,3,Título
1,1,Libertadores
4,4,Sul-Americana
2,2,Limbo
0,0,Rebaixamento


In [133]:
tabela

,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG,cluster
0,PAL,20,10,7,3,36.0,17.0,37,19.0,3
1,AME,20,2,5,13,20.0,44.0,11,-24.0,0
2,CAP,20,9,5,6,29.0,23.0,32,6.0,1
3,BOT,20,15,3,2,35.0,11.0,48,24.0,3
4,RED,20,8,8,4,27.0,21.0,32,6.0,1
5,FOR,20,8,5,7,22.0,19.0,29,3.0,4
6,CAM,20,7,6,7,22.0,18.0,27,4.0,4
7,COR,20,6,7,7,21.0,22.0,25,-1.0,4
8,FLA,20,10,5,5,34.0,26.0,35,8.0,1
9,GRE,20,10,4,6,29.0,25.0,34,4.0,1


In [134]:
tabela_sort = tabela.sort_values(by=['2-P','4-V','9-SG'], ascending=False)
tabela_sort.style.background_gradient(cmap='Blues')
# tabela_sort.style.background_gradient(cmap='Blues').format("{:.2f}").render()


,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG,cluster
3,BOT,20,15,3,2,35.000000,11.000000,48,24.000000,3
0,PAL,20,10,7,3,36.000000,17.000000,37,19.000000,3
8,FLA,20,10,5,5,34.000000,26.000000,35,8.000000,1
10,FLU,20,10,4,6,28.000000,20.000000,34,8.000000,1
9,GRE,20,10,4,6,29.000000,25.000000,34,4.000000,1
2,CAP,20,9,5,6,29.000000,23.000000,32,6.000000,1
4,RED,20,8,8,4,27.000000,21.000000,32,6.000000,1
5,FOR,20,8,5,7,22.000000,19.000000,29,3.000000,4
11,CUI,20,8,4,8,21.000000,23.000000,28,-2.000000,4
12,SAO,20,7,7,6,24.000000,19.000000,28,5.000000,4


In [135]:
#NORMALIZAR A TABELA E COLORIR...

#grafico com pontuacao original e dos cluster

#grafico dos cluster

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler


In [136]:
df_data = tabela[[
    'cluster',              
    '2-P',              
    '4-V',
    '5-E',
    '6-D',
    '9-SG'
]]

In [137]:
X_Train = df_data.drop(columns=['cluster'], axis=1)
X_Test = df_data.drop(columns=['cluster'], axis=1)
y_Train = df_data['cluster']
y_Test = df_data['cluster']

In [138]:
sc_x = StandardScaler()
X_Train = sc_x.fit_transform(X_Train)
X_Test = sc_x.fit_transform(X_Test)

In [139]:
logreg = LogisticRegression(solver="lbfgs", max_iter=500)
logreg.fit(X_Train, y_Train)
pred_logreg = logreg.predict(X_Test)
pred_proba = logreg.predict_proba(X_Test)

In [140]:
tabela["cluster_pred"] = pred_logreg

In [141]:
lista_proba = pred_proba.tolist()

data = []  # Lista para armazenar os dicionários de dados

index = 0
for proba in lista_proba:
    for i in range(0, len(proba)):
        new_row = {"index": index, "prob": i, "valor": round(proba[i], 4)}
        data.append(new_row)
    index += 1

df_prob = pd.DataFrame(data)  # Cria o DataFrame a partir da lista de dicionários
df_prob = df_prob.pivot_table(
    index="index", columns="prob", values="valor", aggfunc="sum"
)
df_prob = df_prob.reset_index()
df_prob = df_prob.set_index("index")


In [142]:
df_prob = df_prob.rename(columns={
    0.0: 'cl_o',
    1.0: 'cl_1',
    2.0: 'cl_2',
    3.0: 'cl_3',
    4.0: 'cl_4'
})

In [143]:
tabela = pd.merge(tabela, df_prob, left_index=True, right_index=True)
tabela

,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG,cluster,cluster_pred,cl_o,cl_1,cl_2,cl_3,cl_4
0,PAL,20,10,7,3,36.0,17.0,37,19.0,3,3,0.0000,0.3583,0.0055,0.4957,0.1405
1,AME,20,2,5,13,20.0,44.0,11,-24.0,0,0,0.8482,0.0002,0.1418,0.0000,0.0098
2,CAP,20,9,5,6,29.0,23.0,32,6.0,1,1,0.0051,0.5040,0.0699,0.0940,0.3271
3,BOT,20,15,3,2,35.0,11.0,48,24.0,3,3,0.0000,0.2790,0.0001,0.7171,0.0039
4,RED,20,8,8,4,27.0,21.0,32,6.0,1,4,0.0009,0.3896,0.0696,0.1321,0.4077
5,FOR,20,8,5,7,22.0,19.0,29,3.0,4,4,0.0203,0.3472,0.1527,0.0380,0.4417
6,CAM,20,7,6,7,22.0,18.0,27,4.0,4,4,0.0197,0.2126,0.1857,0.0261,0.5559
7,COR,20,6,7,7,21.0,22.0,25,-1.0,4,4,0.0285,0.1245,0.3026,0.0100,0.5345
8,FLA,20,10,5,5,34.0,26.0,35,8.0,1,1,0.0011,0.6033,0.0279,0.1754,0.1924
9,GRE,20,10,4,6,29.0,25.0,34,4.0,1,1,0.0045,0.6505,0.0480,0.0992,0.1978


In [144]:
df_pred = tabela[[
    '1-Time',
    'cl_o',
    'cl_1',
    'cl_2',
    'cl_3',
    'cl_4',
]]
df_pred = df_pred.copy()
df_pred

,1-Time,cl_o,cl_1,cl_2,cl_3,cl_4
0,PAL,0.0000,0.3583,0.0055,0.4957,0.1405
1,AME,0.8482,0.0002,0.1418,0.0000,0.0098
2,CAP,0.0051,0.5040,0.0699,0.0940,0.3271
3,BOT,0.0000,0.2790,0.0001,0.7171,0.0039
4,RED,0.0009,0.3896,0.0696,0.1321,0.4077
5,FOR,0.0203,0.3472,0.1527,0.0380,0.4417
6,CAM,0.0197,0.2126,0.1857,0.0261,0.5559
7,COR,0.0285,0.1245,0.3026,0.0100,0.5345
8,FLA,0.0011,0.6033,0.0279,0.1754,0.1924
9,GRE,0.0045,0.6505,0.0480,0.0992,0.1978


In [145]:
df_pred.style.background_gradient(cmap='Blues', axis=1)

,1-Time,cl_o,cl_1,cl_2,cl_3,cl_4
0,PAL,0.000000,0.358300,0.005500,0.495700,0.140500
1,AME,0.848200,0.000200,0.141800,0.000000,0.009800
2,CAP,0.005100,0.504000,0.069900,0.094000,0.327100
3,BOT,0.000000,0.279000,0.000100,0.717100,0.003900
4,RED,0.000900,0.389600,0.069600,0.132100,0.407700
5,FOR,0.020300,0.347200,0.152700,0.038000,0.441700
6,CAM,0.019700,0.212600,0.185700,0.026100,0.555900
7,COR,0.028500,0.124500,0.302600,0.010000,0.534500
8,FLA,0.001100,0.603300,0.027900,0.175400,0.192400
9,GRE,0.004500,0.650500,0.048000,0.099200,0.197800


In [146]:
rodadas = brasileirao['Rodada'].unique()
times = brasileirao['Mandante'].unique()
pontuacao = pd.DataFrame()
data = []  # Lista para armazenar os dicionários de dados

for rodada in rodadas:
    for time in times:
        resultado = brasileirao[(brasileirao['Rodada'] == rodada) & ((brasileirao['Mandante'] == time) | (brasileirao['Visitante'] == time))]
        if len(resultado) > 0:
            resultado = resultado.reset_index()
            if resultado['Mandante'][0] == time:
                if resultado['Score_m'][0] > resultado['Score_v'][0]:
                    pontos = 3
                elif resultado['Score_m'][0] == resultado['Score_v'][0]:
                    pontos = 1
                else:
                    pontos = 0
            else:                  
                if resultado['Score_v'][0] > resultado['Score_m'][0]:
                    pontos = 3
                elif resultado['Score_v'][0] == resultado['Score_m'][0]:
                    pontos = 1
                else:
                    pontos = 0
            new_row = {
                'time': time,
                'rodada': rodada,
                'pontos': pontos
            }        
            data.append(new_row)  # Adiciona o dicionário à lista de dados

pontuacao = pd.DataFrame(data)  # Cria o DataFrame a partir da lista de dicionários

pt_pontuacao = pontuacao.pivot_table(index='rodada', columns='time', values='pontos', aggfunc='sum')
pt_pontuacao_cum = pt_pontuacao.cumsum()
pt_pontuacao_cum

time,AME,BAH,BOT,CAM,CAP,CFC,COR,CRU,CUI,FLA,FLU,FOR,GOI,GRE,INT,PAL,RED,SAN,SAO,VAS
rodada,,,,,,,,,,,,,,,,,,,,
1,0,0,3,0,3,0,3,0,0,3,3,1,0,3,1,3,3,0,0,3
2,0,0,6,1,3,0,3,3,1,3,6,4,3,3,4,4,4,1,3,4
3,0,3,9,4,3,1,3,6,1,3,6,7,3,6,7,7,4,4,4,4
4,0,6,12,4,6,1,4,9,4,3,7,8,3,7,7,10,5,4,7,5
5,1,6,15,7,9,2,4,9,4,6,10,9,3,7,7,13,6,7,8,6
6,1,6,15,10,12,2,5,12,4,9,13,10,6,8,7,14,7,10,9,6
7,4,7,18,13,12,2,5,12,7,12,13,10,7,11,7,15,10,11,12,6
8,4,7,21,14,12,3,8,13,8,13,13,13,7,14,10,16,13,11,15,6
9,7,8,21,17,15,3,8,13,11,16,16,14,7,17,11,19,13,12,15,6


In [147]:
pt_pontuacao_cum = pt_pontuacao_cum.reset_index()
pt_pontuacao_cum

time,rodada,AME,BAH,BOT,CAM,CAP,CFC,COR,CRU,CUI,...,FLU,FOR,GOI,GRE,INT,PAL,RED,SAN,SAO,VAS
0,1,0,0,3,0,3,0,3,0,0,...,3,1,0,3,1,3,3,0,0,3
1,2,0,0,6,1,3,0,3,3,1,...,6,4,3,3,4,4,4,1,3,4
2,3,0,3,9,4,3,1,3,6,1,...,6,7,3,6,7,7,4,4,4,4
3,4,0,6,12,4,6,1,4,9,4,...,7,8,3,7,7,10,5,4,7,5
4,5,1,6,15,7,9,2,4,9,4,...,10,9,3,7,7,13,6,7,8,6
5,6,1,6,15,10,12,2,5,12,4,...,13,10,6,8,7,14,7,10,9,6
6,7,4,7,18,13,12,2,5,12,7,...,13,10,7,11,7,15,10,11,12,6
7,8,4,7,21,14,12,3,8,13,8,...,13,13,7,14,10,16,13,11,15,6
8,9,7,8,21,17,15,3,8,13,11,...,16,14,7,17,11,19,13,12,15,6
9,10,8,9,24,18,16,4,9,14,12,...,17,14,8,17,14,22,14,13,15,6


In [148]:
from sklearn.linear_model import LinearRegression

In [149]:
colunas = pt_pontuacao_cum.columns
df_regressao = pd.DataFrame()
data_regressao = []  # Lista para armazenar os dicionários de resultados de regressão

for coluna in colunas:
    if coluna != 'rodada':
        X = pt_pontuacao_cum['rodada'].values.reshape(-1, 1)
        y = pt_pontuacao_cum[coluna].values.reshape(-1, 1)
        regressor = LinearRegression()
        regressor.fit(X, y)
        A = regressor.intercept_[0]
        B = regressor.coef_[0][0]
        x = A + (B * 38)
        new_row_regressao = {
            'time': coluna,
            'intercept': round(A, 2),
            'slope': round(B, 2),
            'pontuacao_final': round(x, 2)
        }
        data_regressao.append(new_row_regressao)

df_regressao = pd.DataFrame(data_regressao)
df_regressao.sort_values(by='pontuacao_final', ascending=False)

,time,intercept,slope,pontuacao_final
2,BOT,1.39,2.40,92.45
9,FLA,-1.31,1.85,69.12
13,GRE,-0.30,1.81,68.64
15,PAL,2.95,1.62,64.66
16,RED,-1.13,1.70,63.58
10,FLU,1.77,1.55,60.86
4,CAP,0.32,1.56,59.43
18,SAO,0.73,1.47,56.73
8,CUI,-3.75,1.58,56.33
3,CAM,1.02,1.36,52.88


In [150]:
df_regressao = df_regressao[['time','slope']]
df_regressao

,time,slope
0,AME,0.72
1,BAH,0.96
2,BOT,2.40
3,CAM,1.36
4,CAP,1.56
5,CFC,0.87
6,COR,1.20
7,CRU,1.22
8,CUI,1.58
9,FLA,1.85


In [151]:
tabela = tabela.merge(df_regressao, left_on='1-Time', right_on='time', how='left')
tabela

,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG,cluster,cluster_pred,cl_o,cl_1,cl_2,cl_3,cl_4,time,slope
0,PAL,20,10,7,3,36.0,17.0,37,19.0,3,3,0.0000,0.3583,0.0055,0.4957,0.1405,PAL,1.62
1,AME,20,2,5,13,20.0,44.0,11,-24.0,0,0,0.8482,0.0002,0.1418,0.0000,0.0098,AME,0.72
2,CAP,20,9,5,6,29.0,23.0,32,6.0,1,1,0.0051,0.5040,0.0699,0.0940,0.3271,CAP,1.56
3,BOT,20,15,3,2,35.0,11.0,48,24.0,3,3,0.0000,0.2790,0.0001,0.7171,0.0039,BOT,2.40
4,RED,20,8,8,4,27.0,21.0,32,6.0,1,4,0.0009,0.3896,0.0696,0.1321,0.4077,RED,1.70
5,FOR,20,8,5,7,22.0,19.0,29,3.0,4,4,0.0203,0.3472,0.1527,0.0380,0.4417,FOR,1.32
6,CAM,20,7,6,7,22.0,18.0,27,4.0,4,4,0.0197,0.2126,0.1857,0.0261,0.5559,CAM,1.36
7,COR,20,6,7,7,21.0,22.0,25,-1.0,4,4,0.0285,0.1245,0.3026,0.0100,0.5345,COR,1.20
8,FLA,20,10,5,5,34.0,26.0,35,8.0,1,1,0.0011,0.6033,0.0279,0.1754,0.1924,FLA,1.85
9,GRE,20,10,4,6,29.0,25.0,34,4.0,1,1,0.0045,0.6505,0.0480,0.0992,0.1978,GRE,1.81


In [152]:
from sklearn.cluster import KMeans
df_data = tabela[[
    '2-P',              
    '4-V',
    '5-E',
    '6-D',
    '9-SG',
    'slope'
]]
kmeans = KMeans(n_clusters=5, random_state=0).fit(df_data)
tabela['cluster'] = kmeans.labels_
tabela['c_p'] = kmeans.cluster_centers_[kmeans.labels_,0]
tabela['c_v'] = kmeans.cluster_centers_[kmeans.labels_,1]
tabela['c_e'] = kmeans.cluster_centers_[kmeans.labels_,2]
tabela['c_d'] = kmeans.cluster_centers_[kmeans.labels_,3]
tabela['c_sg'] = kmeans.cluster_centers_[kmeans.labels_,4]
tabela['c_slope'] = kmeans.cluster_centers_[kmeans.labels_,5]
tabela_sort = tabela.sort_values(by=['2-P','4-V','9-SG'], ascending=False)
tabela_sort.style.background_gradient(cmap='Blues')


c:\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG,cluster,cluster_pred,cl_o,cl_1,cl_2,cl_3,cl_4,time,slope,c_p,c_v,c_e,c_d,c_sg,c_slope
3,BOT,20,15,3,2,35.000000,11.000000,48,24.000000,3,3,0.000000,0.279000,0.000100,0.717100,0.003900,BOT,2.400000,42.500000,12.500000,5.000000,2.500000,21.500000,2.010000
0,PAL,20,10,7,3,36.000000,17.000000,37,19.000000,3,3,0.000000,0.358300,0.005500,0.495700,0.140500,PAL,1.620000,42.500000,12.500000,5.000000,2.500000,21.500000,2.010000
8,FLA,20,10,5,5,34.000000,26.000000,35,8.000000,1,1,0.001100,0.603300,0.027900,0.175400,0.192400,FLA,1.850000,33.400000,9.400000,5.200000,5.400000,6.400000,1.694000
10,FLU,20,10,4,6,28.000000,20.000000,34,8.000000,1,1,0.003100,0.611400,0.036100,0.133400,0.216000,FLU,1.550000,33.400000,9.400000,5.200000,5.400000,6.400000,1.694000
9,GRE,20,10,4,6,29.000000,25.000000,34,4.000000,1,1,0.004500,0.650500,0.048000,0.099200,0.197800,GRE,1.810000,33.400000,9.400000,5.200000,5.400000,6.400000,1.694000
2,CAP,20,9,5,6,29.000000,23.000000,32,6.000000,1,1,0.005100,0.504000,0.069900,0.094000,0.327100,CAP,1.560000,33.400000,9.400000,5.200000,5.400000,6.400000,1.694000
4,RED,20,8,8,4,27.000000,21.000000,32,6.000000,1,4,0.000900,0.389600,0.069600,0.132100,0.407700,RED,1.700000,33.400000,9.400000,5.200000,5.400000,6.400000,1.694000
5,FOR,20,8,5,7,22.000000,19.000000,29,3.000000,4,4,0.020300,0.347200,0.152700,0.038000,0.441700,FOR,1.320000,27.000000,7.000000,6.000000,7.000000,2.000000,1.358333
11,CUI,20,8,4,8,21.000000,23.000000,28,-2.000000,4,4,0.072700,0.312700,0.232700,0.016400,0.365600,CUI,1.580000,27.000000,7.000000,6.000000,7.000000,2.000000,1.358333
12,SAO,20,7,7,6,24.000000,19.000000,28,5.000000,4,4,0.007300,0.232200,0.150200,0.041500,0.568800,SAO,1.470000,27.000000,7.000000,6.000000,7.000000,2.000000,1.358333


In [155]:
pontuacao_2 = pt_pontuacao_cum[pt_pontuacao_cum['rodada'] > 19]
pontuacao_2

time,rodada,AME,BAH,BOT,CAM,CAP,CFC,COR,CRU,CUI,...,FLU,FOR,GOI,GRE,INT,PAL,RED,SAN,SAO,VAS
10,11,8,12,27,19,16,4,12,14,12,...,18,17,11,20,17,22,17,13,18,6
11,12,8,12,30,19,19,4,12,17,12,...,21,20,11,23,20,22,20,13,18,9
12,13,9,12,33,20,20,7,12,18,15,...,21,20,11,26,21,23,23,13,21,9
13,14,9,13,36,20,20,10,15,21,16,...,24,23,11,26,21,24,24,16,22,9
14,15,10,13,39,21,23,11,16,22,19,...,25,23,12,27,22,25,24,16,25,10
15,16,11,14,40,21,26,14,17,22,22,...,25,23,15,30,23,28,25,17,25,10
16,17,11,15,43,21,27,14,20,23,25,...,28,23,16,31,23,31,28,17,26,10
17,18,11,18,44,24,28,14,21,24,28,...,31,23,19,31,24,31,31,18,26,13
18,19,11,18,47,27,31,14,24,24,28,...,31,26,22,34,24,34,32,18,27,14
19,20,11,21,48,27,32,14,25,25,28,...,34,29,23,34,24,37,32,21,28,17


In [156]:
colunas = pontuacao_2.columns
df_regressao = pd.DataFrame()
colunas = pontuacao_2.columns
regressao_data = []

for coluna in colunas:
    if coluna != 'rodada':
        X = pontuacao_2['rodada'].values.reshape(-1, 1)
        y = pontuacao_2[coluna].values.reshape(-1, 1)
        regressor = LinearRegression()
        regressor.fit(X, y)
        A = regressor.intercept_[0]
        B = regressor.coef_[0][0]
        x = A + (B * 38)
        regressao_data.append({
            'time': coluna,
            'intercept': round(A, 2),
            'slope': round(B, 2),
            'pontuacao_final': round(x, 2)
        })

df_regressao = pd.DataFrame(regressao_data)
df_regressao.sort_values(by='pontuacao_final', ascending=False)

,time,intercept,slope,pontuacao_final
2,BOT,2.53,2.33,91.20
8,CUI,-12.47,2.13,68.36
15,PAL,1.12,1.72,66.29
4,CAP,-3.23,1.77,64.02
10,FLU,-0.13,1.67,63.44
9,FLA,3.18,1.58,63.05
16,RED,0.05,1.65,62.69
13,GRE,5.09,1.49,61.75
6,COR,-7.21,1.59,53.13
18,SAO,5.75,1.15,49.51
